# Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import time
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [0]:
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [0]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [0]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


##### Выводы по первому взгляду на данные:

Таблица состоит из следующих столбцов:
 - DateCrawled — дата скачивания анкеты из базы;
 - VehicleType — тип автомобильного кузова;
 - RegistrationYear — год регистрации автомобиля;
 - Gearbox — тип коробки передач;
 - Power — мощность (л. с.);
 - Model — модель автомобиля;
 - Kilometer — пробег (км);
 - RegistrationMonth — месяц регистрации автомобиля;
 - FuelType — тип топлива;
 - Brand — марка автомобиля;
 - NotRepaired — была машина в ремонте или нет;
 - DateCreated — дата создания анкеты;
 - NumberOfPictures — количество фотографий автомобиля;
 - PostalCode — почтовый индекс владельца анкеты (пользователя);
 - LastSeen — дата последней активности пользователя;
 - Price — цена (евро) - является целевым признаком.

В некоторых столбцах (VehicleType, Gearbox, Model, FuelType, NotRepaired) присутсвует большое количество пропусков. Столбец NumberOfPictures не содержит значений, во всех строках нули. Столбец RegistrationYear содержит аномальные значения (1000, 9999). Столбец Power также содержит аномальные значения (0 лошадиных сил, 20000 л.с.). Цена в 0 евро также является выбросом. Столбец RegistrationMonth содержит значения от 0 до 12, тогда как месяцев всего 12. Вероятно, что 0 - это аномальное значение.

Проверим наличие и удалим явные дубликаты:

In [0]:
print('Количество дубликатов:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

Количество дубликатов: 4


Посмотрим количество машин со стоимостью 0 евро и еще несколько минимальных значений:

In [0]:
print('Количество объявлений со стоимостью 0 евро:', 
         data[data['Price']==0]['Price'].value_counts().sum())
for i in [10, 50, 100]:
    print('Количество объявлений со стоимостью, меньшей', i, 'евро :', 
         data[data['Price']<i]['Price'].value_counts().sum())

Количество объявлений со стоимостью 0 евро: 10772
Количество объявлений со стоимостью, меньшей 10 евро : 12028
Количество объявлений со стоимостью, меньшей 50 евро : 12423
Количество объявлений со стоимостью, меньшей 100 евро : 13314


Как видно, для машины со стоимостью меньше 100 евро в таблице практически не встречаются. Эту стоимость и будем считать порогом для цены. Заменим цены ниже на пропуск и удалим значения из таблицы.

In [0]:
data['Price'] = np.where(data['Price']<100, np.nan, data['Price'])
data = data.dropna(subset=['Price'])

Посмотрим на уникальные значения, встречающиеся в столбце FuelType:

In [0]:
pd.Series(data['FuelType'].unique())

0      petrol
1    gasoline
2         NaN
3         lpg
4       other
5      hybrid
6         cng
7    electric
dtype: object

Так как petrol и gasoline означают один и тот же вид топлива, произведем следующую замену:

In [0]:
data['FuelType'] = data['FuelType'].replace('gasoline', 'petrol')

Мощность возьмем в пределах 8 - 500 л.с.

In [0]:
data['Power'] = np.where(((data['Power'] >= 500) | (data['Power'] < 8)),
                         np.nan, data['Power'])

Года регистрации возьмем в пределах 1910-2019. (Такой диапазон выбран потому, что раньше 1910 года значения аномальные, а 2019 - таблица по этот год):

In [0]:
data['RegistrationYear'] = np.where((data['RegistrationYear'] > 2019) | (data['RegistrationYear'] < 1910),
                                    np.nan, data['RegistrationYear'])

"Нулевой" месяц заменим на пропуски:

In [0]:
data['RegistrationMonth'] = data['RegistrationMonth'].replace(0, np.nan)

Для заполнения столбца VehicleType создадим словарь пар: "модель - самый часто встречающайся тип кузова" и заполним пропуски этими значенями:

In [0]:
vehicle_type_map = data[['Model']].dropna().drop_duplicates().reset_index(drop=True)
for i in range(len(vehicle_type_map)):
    vehicle_type_map.loc[i, 'VehicleType'] = (data[data['Model'] == vehicle_type_map.loc[i, 'Model']]['VehicleType']
        .value_counts().reset_index().iloc[0, 0])

vehicle_type_map = vehicle_type_map.set_index('Model').to_dict()['VehicleType']
vehicle_type_map[np.nan] = np.nan

data['VehicleType'] = data.apply(lambda row: vehicle_type_map[row['Model']]
                                if row['VehicleType'] in (None, np.nan)
                                else row['VehicleType'], axis=1)

Удалим столбцы:

In [0]:
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', 'NumberOfPictures'], axis=1)

Окончательно получим две таблицы: в первой удалим пропущенные значени, во второй заполним их маркером "-1".

In [0]:
data_drop = data.dropna()
data_marker = data.fillna('-1')

Выведем размеры получившихся таблиц:

In [0]:
print('data_drop:', data_drop.shape)
print('data_marker:', data_marker.shape)

data_drop: (231573, 11)
data_marker: (341051, 11)


Предобработка закончена, приступаем к моделям.

# 2. Обучение моделей

Запишем в списки названия категориальных и количественных столбцов:

In [0]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
numeric_features = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

### 2.1. Модели по таблице с удаленными пропусками (dropna)

Подготовим данные для обучения из меньшей таблицы (с дропнутыми пропусками). Произведем кодирование категориальных признаков с помощью OrdinalEncoder и стандартизирование количественных с помощью StandardScaler.

In [0]:
data_ordinal = data_drop.copy()
encoder = OrdinalEncoder()
data_ordinal[cat_features] = encoder.fit_transform(data_ordinal[cat_features])

df_train, df_temp = train_test_split(data_ordinal, test_size=0.2, random_state=12345)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=12345)

features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

scaler = StandardScaler()
scaler.fit(features_train[numeric_features])
features_train[numeric_features] = scaler.transform(features_train[numeric_features])
features_valid[numeric_features] = scaler.transform(features_valid[numeric_features])
features_test[numeric_features] = scaler.transform(features_test[numeric_features])

# Выведем информацию о размерах выборок, чтобы убедиться в правильной разбивке
print('Количество строк и столбцов в обучающей выборке:', df_train.shape)
print('Количество строк и столбцов в валидационной выборке:', df_valid.shape)
print('Количество строк и столбцов в тестовой выборке:', df_test.shape)

Количество строк и столбцов в обучающей выборке: (185258, 11)
Количество строк и столбцов в валидационной выборке: (23157, 11)
Количество строк и столбцов в тестовой выборке: (23158, 11)


Посчитаем RMSE для константной модели, где все значения - это среднее цены в обучающей выборке:

In [0]:
%%time
print('Константная модель')

pred_mean_train = pd.Series(target_train.mean(), index=target_train.index)
pred_mean_valid = pd.Series(target_train.mean(), index=target_valid.index)
pred_mean_test = pd.Series(target_train.mean(), index=target_test.index)

rmse_train = mean_squared_error(target_train, pred_mean_train) ** 0.5
print('rmse_train:', rmse_train)

rmse_valid = mean_squared_error(target_valid, pred_mean_valid) ** 0.5
print('rmse_valid:', rmse_valid)

rmse_test = mean_squared_error(target_test, pred_mean_test) ** 0.5
print('rmse_test:', rmse_test)

Константная модель
rmse_train: 4729.041121133357
rmse_valid: 4697.6205174897905
rmse_test: 4740.37542076567
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 8.52 ms


Обучим линейную регрессию:

In [0]:
data_ohe = pd.get_dummies(data_drop, drop_first=True)

df_train_ohe, df_temp_ohe = train_test_split(data_ohe, test_size=0.2, random_state=12345)
df_valid_ohe, df_test_ohe = train_test_split(df_temp_ohe, test_size=0.5, random_state=12345)

features_train_ohe = df_train_ohe.drop(['Price'], axis=1)
target_train_ohe = df_train_ohe['Price']
features_valid_ohe = df_valid_ohe.drop(['Price'], axis=1)
target_valid_ohe = df_valid_ohe['Price']
features_test_ohe = df_test_ohe.drop(['Price'], axis=1)
target_test_ohe = df_test_ohe['Price']

scaler_ohe = StandardScaler()
scaler_ohe.fit(features_train_ohe[numeric_features])
features_train_ohe[numeric_features] = scaler_ohe.transform(features_train_ohe[numeric_features])
features_valid_ohe[numeric_features] = scaler_ohe.transform(features_valid_ohe[numeric_features])
features_test_ohe[numeric_features] = scaler_ohe.transform(features_test_ohe[numeric_features])

In [0]:
%%time

model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)

CPU times: user 13.3 s, sys: 4.3 s, total: 17.6 s
Wall time: 17.6 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
print('Линейная регрессия')

pred_lr_train = model_lr.predict(features_train_ohe)
rmse_lr_train = mean_squared_error(target_train_ohe, pred_lr_train) ** 0.5
print('rmse_lr_train:', rmse_lr_train)

pred_lr_valid = model_lr.predict(features_valid_ohe)
rmse_lr_valid = mean_squared_error(target_valid_ohe, pred_lr_valid) ** 0.5
print('rmse_lr_valid:', rmse_lr_valid)

pred_lr_test = model_lr.predict(features_test_ohe)
rmse_lr_test = mean_squared_error(target_test_ohe, pred_lr_test) ** 0.5
print('rmse_lr_test:', rmse_lr_test)

Линейная регрессия
rmse_lr_train: 2616.856119427848
rmse_lr_valid: 2582.889719519657
rmse_lr_test: 2667.1884202259594


Обучим модель случайного леса:

In [0]:
%%time

model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=12345)
model_rf.fit(features_train, target_train)

CPU times: user 32.2 s, sys: 0 ns, total: 32.2 s
Wall time: 32.5 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=12345,
                      verbose=0, warm_start=False)

In [0]:
print('Случайный лес')

pred_rf_train = model_rf.predict(features_train)
rmse_rf_train = mean_squared_error(target_train, pred_rf_train) ** 0.5
print('rmse_rf_train:', rmse_rf_train)

pred_rf_valid = model_rf.predict(features_valid)
rmse_rf_valid = mean_squared_error(target_valid, pred_rf_valid) ** 0.5
print('rmse_rf_valid:', rmse_rf_valid)

pred_rf_test = model_rf.predict(features_test)
rmse_rf_test = mean_squared_error(target_test, pred_rf_test) ** 0.5
print('rmse_rf_test:', rmse_rf_test)

Случайный лес
rmse_rf_train: 1822.4434047604977
rmse_rf_valid: 1867.0760338200414
rmse_rf_test: 1905.0200857424418


Обучим модель градиентного бустинга lightgbm со всеми стандартным гиперпараметрами и посчитаем RMSE:

In [0]:
%%time
model1 = lgb.LGBMRegressor()
model1.fit(features_train, target_train)

CPU times: user 7.21 s, sys: 0 ns, total: 7.21 s
Wall time: 7.3 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
print('"Стандартный" бустинг')

pred1_train = model1.predict(features_train)
rmse1_train = mean_squared_error(target_train, pred1_train) ** 0.5
print('rmse1_train:', rmse1_train)

pred1_valid = model1.predict(features_valid)
rmse1_valid = mean_squared_error(target_valid, pred1_valid) ** 0.5
print('rmse1_valid:', rmse1_valid)

pred1_test = model1.predict(features_test)
rmse1_test = mean_squared_error(target_test, pred1_test) ** 0.5
print('rmse1_test:', rmse1_test)

"Стандартный" бустинг
rmse1_train: 1670.2971142307638
rmse1_valid: 1682.1302255331964
rmse1_test: 1716.6311286224563


Обучим модель градиентного бустинга lightgbm с настроеными гиперпараметрами и посчитаем RMSE:

In [0]:
lgb_train = lgb.Dataset(features_train, target_train, categorical_feature=cat_features, free_raw_data=False)
lgb_eval = lgb.Dataset(features_valid, target_valid,
                       reference=lgb_train, categorical_feature=cat_features, free_raw_data=False)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 400,  # 31
    'min_data_in_leaf': 20,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'max_depth': 8}

In [0]:
%%time
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=15,
                verbose_eval=False)

CPU times: user 1min 20s, sys: 640 ms, total: 1min 20s
Wall time: 1min 21s


In [0]:
print('Настроеный бустинг')
pred2_train = gbm.predict(features_train, num_iteration=gbm.best_iteration)
rmse2_train = mean_squared_error(target_train, pred2_train) ** 0.5
print('rmse2_train:', rmse2_train)

pred2_valid = gbm.predict(features_valid, num_iteration=gbm.best_iteration)
rmse2_valid = mean_squared_error(target_valid, pred2_valid) ** 0.5
print('rmse2_valid:', rmse2_valid)

pred2_test = gbm.predict(features_test, num_iteration=gbm.best_iteration)
rmse2_test = mean_squared_error(target_test, pred2_test) ** 0.5
print('rmse2_test:', rmse2_test)

Настроеный бустинг
rmse2_train: 1345.8350220241487
rmse2_valid: 1511.0527131424076
rmse2_test: 1545.4070681497196


Обучим модель градиентного бустинга CatBoost и посчитаем RMSE

In [0]:
data_catboost = data_drop.copy()

df_train_catboost, df_temp_catboost = train_test_split(data_catboost, test_size=0.2, random_state=12345)
df_valid_catboost, df_test_catboost = train_test_split(df_temp_catboost, test_size=0.5, random_state=12345)

features_train_catboost = df_train_catboost.drop(['Price'], axis=1)
target_train_catboost = df_train_catboost['Price']
features_valid_catboost = df_valid_catboost.drop(['Price'], axis=1)
target_valid_catboost = df_valid_catboost['Price']
features_test_catboost = df_test_catboost.drop(['Price'], axis=1)
target_test_catboost = df_test_catboost['Price']

scaler_catboost = StandardScaler()
scaler_catboost.fit(features_train_catboost[numeric_features])
features_train_catboost[numeric_features] = scaler_catboost.transform(features_train_catboost[numeric_features])
features_valid_catboost[numeric_features] = scaler_catboost.transform(features_valid_catboost[numeric_features])
features_test_catboost[numeric_features] = scaler_catboost.transform(features_test_catboost[numeric_features])

# Выведем информацию о размерах выборок, чтобы убедиться в правильной разбивке
print('Количество строк и столбцов в обучающей выборке:', df_train_catboost.shape)
print('Количество строк и столбцов в валидационной выборке:', df_valid_catboost.shape)
print('Количество строк и столбцов в тестовой выборке:', df_test_catboost.shape)

Количество строк и столбцов в обучающей выборке: (185258, 11)
Количество строк и столбцов в валидационной выборке: (23157, 11)
Количество строк и столбцов в тестовой выборке: (23158, 11)


In [0]:
%%time
model_catboost = CatBoostRegressor(iterations=1000)
model_catboost.fit(features_train_catboost, target_train_catboost, cat_features=cat_features, verbose=100)

0:	learn: 4632.2045750	total: 733ms	remaining: 12m 12s
100:	learn: 1994.5821735	total: 50.7s	remaining: 7m 31s
200:	learn: 1831.6385981	total: 1m 38s	remaining: 6m 30s
300:	learn: 1774.4235756	total: 2m 24s	remaining: 5m 34s
400:	learn: 1739.0412083	total: 3m 11s	remaining: 4m 46s
500:	learn: 1711.4828931	total: 3m 59s	remaining: 3m 58s
600:	learn: 1691.1541604	total: 4m 48s	remaining: 3m 11s
700:	learn: 1674.0451037	total: 5m 36s	remaining: 2m 23s
800:	learn: 1660.9261601	total: 6m 24s	remaining: 1m 35s
900:	learn: 1648.2955421	total: 7m 11s	remaining: 47.4s
999:	learn: 1639.1584123	total: 7m 59s	remaining: 0us
CPU times: user 7min 3s, sys: 57.7 s, total: 8min 1s
Wall time: 8min 8s


In [0]:
print('CatBoost')
pred_catboost_train = model_catboost.predict(features_train_catboost)
rmse_catboost_train = mean_squared_error(target_train_catboost, pred_catboost_train) ** 0.5
print('rmse_catboost_train:', rmse_catboost_train)

pred_catboost_valid = model_catboost.predict(features_valid_catboost)
rmse_catboost_valid = mean_squared_error(target_valid_catboost, pred_catboost_valid) ** 0.5
print('rmse_catboost_valid:', rmse_catboost_valid)

pred_catboost_test = model_catboost.predict(features_test_catboost)
rmse_catboost_test = mean_squared_error(target_test_catboost, pred_catboost_test) ** 0.5
print('rmse_catboost_test:', rmse_catboost_test)

CatBoost
rmse_catboost_train: 1634.108808761981
rmse_catboost_valid: 1645.4375149796986
rmse_catboost_test: 1680.465497929632


### 2.2. Модели по таблице с  пропусками, заполненными маркером "-1" (fillna)

Подготовим данные для обучения. Произведем кодирование категориальных признаков с помощью OrdinalEncoder и стандартизирование количественных с помощью StandardScaler. Далее обучим "стандартный" и настроенный бустинг, аналогично предыдущему пункту. Линейную регрессию и случайный лес обучать не будем, так они дают больший RMSE. CatBoost тоже не будем использовать, так как он обучается долго.

In [0]:
data_ordinal_marker = data_marker.copy()
encoder = OrdinalEncoder()
data_ordinal_marker[cat_features] = encoder.fit_transform(data_ordinal_marker[cat_features])

for cat in cat_features:
    data_ordinal_marker[cat] = data_ordinal_marker[cat].astype('category')

df_train_marker, df_temp_marker = train_test_split(data_ordinal_marker, test_size=0.2, random_state=12345)
df_valid_marker, df_test_marker = train_test_split(df_temp_marker, test_size=0.5, random_state=12345)

features_train_marker = df_train_marker.drop(['Price'], axis=1)
target_train_marker = df_train_marker['Price']
features_valid_marker = df_valid_marker.drop(['Price'], axis=1)
target_valid_marker = df_valid_marker['Price']
features_test_marker = df_test_marker.drop(['Price'], axis=1)
target_test_marker = df_test_marker['Price']

scaler = StandardScaler()
scaler.fit(features_train_marker[numeric_features])
features_train_marker[numeric_features] = scaler.transform(features_train_marker[numeric_features])
features_valid_marker[numeric_features] = scaler.transform(features_valid_marker[numeric_features])
features_test_marker[numeric_features] = scaler.transform(features_test_marker[numeric_features])

# Выведем информацию о размерах выборок, чтобы убедиться в правильной разбивке
print('Количество строк и столбцов в обучающей выборке:', df_train.shape)
print('Количество строк и столбцов в валидационной выборке:', df_valid.shape)
print('Количество строк и столбцов в тестовой выборке:', df_test.shape)

Количество строк и столбцов в обучающей выборке: (185258, 11)
Количество строк и столбцов в валидационной выборке: (23157, 11)
Количество строк и столбцов в тестовой выборке: (23158, 11)


In [0]:
%%time
model_marker1 = lgb.LGBMRegressor()
model_marker1.fit(features_train_marker, target_train_marker)

CPU times: user 28.1 s, sys: 0 ns, total: 28.1 s
Wall time: 28.3 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
print('"Стандартный" бустинг')

pred_marker1_train = model_marker1.predict(features_train_marker)
rmse_marker1_train = mean_squared_error(target_train_marker, pred_marker1_train) ** 0.5
print('rmse_marker1_train:', rmse_marker1_train)

pred_marker1_valid = model_marker1.predict(features_valid_marker)
rmse_marker1_valid = mean_squared_error(target_valid_marker, pred_marker1_valid) ** 0.5
print('rmse_marker1_valid:', rmse_marker1_valid)

pred_marker1_test = model_marker1.predict(features_test_marker)
rmse_marker1_test = mean_squared_error(target_test_marker, pred_marker1_test) ** 0.5
print('rmse_marker1_test:', rmse_marker1_test)

"Стандартный" бустинг
rmse_marker1_train: 1634.03928468084
rmse_marker1_valid: 1696.686498877347
rmse_marker1_test: 1687.1465761756183


In [0]:
lgb_train_marker = lgb.Dataset(features_train_marker, target_train_marker, 
                               categorical_feature=cat_features, free_raw_data=False)
lgb_eval_marker = lgb.Dataset(features_valid_marker, target_valid_marker,
                       reference=lgb_train_marker, categorical_feature=cat_features, free_raw_data=False)
params_marker = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 400,  # 31
    'min_data_in_leaf': 20,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'max_depth': 8}

In [0]:
%%time
gbm_marker = lgb.train(params_marker,
                lgb_train_marker,
                num_boost_round=5000,
                valid_sets=lgb_eval_marker,
                early_stopping_rounds=15,
                verbose_eval=False)

CPU times: user 2min 16s, sys: 0 ns, total: 2min 16s
Wall time: 2min 18s


In [0]:
print('Настроеный бустинг')
pred_marker_train = gbm_marker.predict(features_train_marker, num_iteration=gbm_marker.best_iteration)
rmse_marker_train = mean_squared_error(target_train_marker, pred_marker_train) ** 0.5
print('rmse_marker_train:', rmse_marker_train)

pred_marker_valid = gbm_marker.predict(features_valid_marker, num_iteration=gbm_marker.best_iteration)
rmse_marker_valid = mean_squared_error(target_valid_marker, pred_marker_valid) ** 0.5
print('rmse_marker_valid:', rmse_marker_valid)

pred_marker_test = gbm_marker.predict(features_test_marker, num_iteration=gbm_marker.best_iteration)
rmse_marker_test = mean_squared_error(target_test_marker, pred_marker_test) ** 0.5
print('rmse_marker_test:', rmse_marker_test)

Настроеный бустинг
rmse_marker_train: 1413.044507966518
rmse_marker_valid: 1596.4072353721392
rmse_marker_test: 1583.2547305642881


Все выводы приведены в пункте 3.

# 3. Анализ моделей

Модель  | RMSE | Время обучения
------------- | ------------- | -------------
Константная модель | 4740 | -
Линейная регрессия | 2667 | 18 сек
Random Forest | 1905 | 31 сек
"Стандартный" lightgbm | 1716 | 8 сек
CatBoost | 1680 | 8 мин
"Настроенный" lightgbm | 1545 | 1 мин 50 сек

- Все построенные модели показали результат выше константной модели.
- Результаты моделей по таблице с удаленными пропусками выше, чем у таблицы с заполнеными маркерами пропусками. Соответсвенно модели из п. 2.2. далее рассматривать не будем (в таблице выше они также не представлены).
- Самая быстрая модель (8 секунд на обучение) - "стандартный" бустинг lightgbm, с RMSE=1716 на тесте. Быстро, но недостаточно точно.
- Лучший результат у "настроенного" бустинга lightgbm: RMSE=1545 c 12,9% переобучением и временем обучения модели 1 мин 50 секунд.
- Линейная регрессия обучалась 18 сек с RMSE=2667, случайный лес 31 секунду с RMSE=1905. По обоим параметрам модели сильно проигрывают "стандартному" lightgbm.
- Модель бустинга CatBoost показала результат лучше "стандартного" бустинга lightgbm, RMSE=1680, переобучение минимально, однако время обучения (8 минут) сводит ее преимущества к нулю.

##### Итого
Оптимальным выбором является "настроенный" бустинг lightgbm с RMSE=1545 и временем обучения модели 1 мин 50 секунд.